## Ejercicio 29
Para los contenidos visibles en wikipedia, cuales son los artículos que tienen la máxima y mínima distancia entre ids de su revisión actual y la anterior. 

## Preparando sesión de Spark

### Spark session y Spark context

In [1]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [2]:
import pandas as pd
import numpy as np 
texts = pd.read_csv('./WikipediaDataset/contents.csv', usecols = ["title", "revision_id", "parent_revision"])
texts.to_parquet('./WikipediaDataset/contentsConRevisiones.parquet')
del texts
#Dataset de contentsConRevisiones.parquet fue creado con las líneas anteriores, y ahora puede ser leído
#de la siguiente forma:
dataframeContents = sqlContext.read.parquet('./WikipediaDataset/contentsConRevisiones.parquet')
rddContents = dataframeContents.rdd

## Preparando y limpiando el dataset

In [7]:
rddContents

MapPartitionsRDD[17] at javaToPython at NativeMethodAccessorImpl.java:0

In [3]:
rddContents1 = rddContents
rddContents1.take(5)

[Row(title='Wikipedia:Artículos solicitados', revision_id=132533307, parent_revision=132516631.0),
 Row(title='Andorra', revision_id=138035057, parent_revision=138027475.0),
 Row(title='Argentina', revision_id=137966826, parent_revision=137825165.0),
 Row(title='Geografía de Andorra', revision_id=137986788, parent_revision=133666383.0),
 Row(title='Demografía de Andorra', revision_id=130869554, parent_revision=129413188.0)]

In [10]:
rddContents1.count()

4132164

In [13]:
rddContents1 = rddContents1.filter(lambda x: ((x[0] != None) and (x[1] != None)))
rddContents1.count()

4132160

In [16]:
rddContents1 = rddContents1.map(lambda x: (x[0], x[1], 0 if (x[2] == None) else x[2]))
rddContents1.take(5)

[('Wikipedia:Artículos solicitados', 132533307, 132516631.0),
 ('Andorra', 138035057, 138027475.0),
 ('Argentina', 137966826, 137825165.0),
 ('Geografía de Andorra', 137986788, 133666383.0),
 ('Demografía de Andorra', 130869554, 129413188.0)]

## Desarrollo

In [18]:
rddConDistancias = rddContents1.map(lambda x: (x[0], abs(x[1] - x[2])))
rddConDistancias.take(5)

[('Wikipedia:Artículos solicitados', 16676.0),
 ('Andorra', 7582.0),
 ('Argentina', 141661.0),
 ('Geografía de Andorra', 4320405.0),
 ('Demografía de Andorra', 1456366.0)]

#### Máxima

In [20]:
filaMaxima = rddConDistancias.reduce(lambda x, y: x if (x[1] > y[1]) else y)
print(filaMaxima)

('Medalla Conmemorativa del 50.º Aniversario de la Milítsia', 138051961)


#### Mínima


In [21]:
filaMinima = rddConDistancias.reduce(lambda x, y: x if (x[1] < y[1]) else y)
print(filaMinima)

('Río Mātukituki', 1.0)


In [22]:
rddConDistancias.filter(lambda x: x[1] == 1).count()

18146

## Conclusión
Se puede observar que la diferencia máxima entre ids de corrección actual y del padre es de 138.051.961, encontrandose en el contenido 'Medalla Conmemorativa del 50.º Aniversario de la Milítsia', mientras que la mínima distancia es de 1, esta estando en el contenido 'Río Mātukituki'. Sin embargo, se demostró que en realidad hay más de 18 mil contenidos que cuentan con la misma distancia mínima. 

## En limpio
A lo largo de la consigna se fue descubriendo los rdds para ir verificando cuáles operaciones hacer y que todo funcione correctamente. Con esto en mente está claro que la consigna puede ser resuelta con menos líneas de código, por lo que se procede a mostrar una solución 'final', con los rdds recién leídos y cargados, saltándose los pasos intermedios de investigación de los rdds.

Las siguientes transformaciones y acciones están hechas sobre el rdds original obtenido al hacer la lectura del archivo correspondiente.

In [26]:
rddDistancias = rddContents.filter(lambda x: ((x[0] != None) and (x[1] != None)))\
.map(lambda x: (x[0], x[1], 0 if (x[2] == None) else x[2])).map(lambda x: (x[0], abs(x[1] - x[2]))).cache()

Se hace un cache() para tener en cache el rddDistancias al que aplicar las dos acciones siguientes. 

#### Máxima

In [27]:
print(rddConDistancias.reduce(lambda x, y: x if (x[1] > y[1]) else y))

('Medalla Conmemorativa del 50.º Aniversario de la Milítsia', 138051961)


#### Mínima

In [28]:
print(rddConDistancias.reduce(lambda x, y: x if (x[1] < y[1]) else y))

('Río Mātukituki', 1.0)
